# Configure The Physical Device

In notebook, we configure a device as the IoT edge device. We use a NC6 Ubuntu Linux VM as the edge device, which is the same Linux VM where you run the current notebook. The goal is to configure the edge device so that it can run [Docker](https://docs.docker.com/v17.12/install/linux/docker-ee/ubuntu), [nvidia-docker](https://github.com/NVIDIA/nvidia-docker), and [IoT Edge runtime](https://docs.microsoft.com/en-us/azure/iot-edge/how-to-install-iot-edge-linux). If another device is used as the edge device, instructions need to be adjusted accordingly. 

In [300]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import time
from dotenv import set_key, get_key, find_dotenv

In [301]:
# get the .env file where all the variables are stored
env_path = find_dotenv(raise_error_if_not_found=True)

In [ ]:
# get previously stored variables
device_connection_string = get_key(env_path, 'device_connection_string')

### Install az-cli iot extension 

In [306]:
accounts = !az account list --all -o tsv
if "Please run \"az login\" to access your accounts." in accounts[0]:
    !az login -o table
else:
    print("Already logged in")

Already logged in


In [ ]:
# install az-cli iot extension 
!az extension add --name azure-cli-iot-ext

### Register Microsoft key and software repository feed

Prepare your device for the IoT Edge runtime installation.

In [326]:
# Install the repository configuration. Replace <release> with 16.04 or 18.04 as appropriate for your release of Ubuntu.
release = !lsb_release -r
release  = release[0].split('\t')[1]
print(release)

16.04


In [327]:
!curl https://packages.microsoft.com/config/ubuntu/$release/prod.list > ./microsoft-prod.list

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    79  100    79    0     0     79      0  0:00:01 --:--:--  0:00:01  4388


In [328]:
# Copy the generated list.
!sudo cp ./microsoft-prod.list /etc/apt/sources.list.d/

In [329]:
#Install Microsoft GPG public key
!curl https://packages.microsoft.com/keys/microsoft.asc | gpg --dearmor > microsoft.gpg
!sudo cp ./microsoft.gpg /etc/apt/trusted.gpg.d/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   983  100   983    0     0    983      0  0:00:01 --:--:--  0:00:01 70214


### Install the Azure IoT Edge Security Daemon

The IoT Edge security daemon provides and maintains security standards on the IoT Edge device. The daemon starts on every boot and bootstraps the device by starting the rest of the IoT Edge runtime.
The installation command also installs the standard version of the iothsmlib if not already present.

In [ ]:
# Perform apt update.
!sudo apt-get update

In [ ]:
# Install the security daemon. The package is installed at /etc/iotedge/.
!sudo apt-get install iotedge -y --no-install-recommends

### Configure the Azure IoT Edge Security 

Configure the IoT Edge runtime to link your physical device with a device identity that exists in an Azure IoT hub.
The daemon can be configured using the configuration file at /etc/iotedge/config.yaml. The file is write-protected by default, you might need elevated permissions to edit it.

In [332]:
# Manual provisioning IoT edge device
!sudo sed -i "s#\(device_connection_string: \).*#\1\"$device_connection_string\"#g" /etc/iotedge/config.yaml

In [334]:
# restart the daemon
!sudo systemctl restart iotedge
time.sleep(20) # Wait 20 seconds for iotedge to restart

In [ ]:
# restart the daemon again
!sudo systemctl restart iotedge

### Verify successful installation

In [338]:
time.sleep(30) # Wait 30 seconds to let the iotedge daemon stable
# check the status of the IoT Edge Daemon
!systemctl status iotedge

● iotedge.service - Azure IoT Edge daemon
   Loaded: loaded (/lib/systemd/system/iotedge.service; enabled; vendor preset: enabled)
   Active: active (running) since Mon 2019-04-08 18:21:30 UTC; 5s ago
     Docs: man:iotedged(8)
 Main PID: 19408 (iotedged)
    Tasks: 19
   Memory: 9.4M
      CPU: 26ms
   CGroup: /system.slice/iotedge.service
           └─19408 /usr/bin/iotedged -c /etc/iotedge/config.yaml

Apr 08 18:21:30 msvole1 iotedged[19408]: 2019-04-08T18:21:30Z [INFO] - Liste....
Apr 08 18:21:30 msvole1 iotedged[19408]: 2019-04-08T18:21:30Z [INFO] - Check...s
Apr 08 18:21:30 msvole1 iotedged[19408]: 2019-04-08T18:21:30Z [INFO] - Edge ....
Apr 08 18:21:30 msvole1 iotedged[19408]: 2019-04-08T18:21:30Z [INFO] - Start....
Apr 08 18:21:30 msvole1 iotedged[19408]: 2019-04-08T18:21:30Z [INFO] - Succe...t
Apr 08 18:21:32 msvole1 iotedged[19408]: 2019-04-08T18:21:32Z [INFO] - Query....
Apr 08 18:21:32 msvole1 iotedged[19408]: 2019-04-08T18:21:32Z [INFO] - [work...)
Apr 08 18:21:32 msvole1 

In [ ]:
# Examine daemon logs
!journalctl -u iotedge --no-pager --no-full

When you run `docker ps` command in the edge device, you should see `edgeAgent` container is up running.

In [340]:
!docker ps

CONTAINER ID        IMAGE                                      COMMAND                   CREATED             STATUS              PORTS               NAMES
5cd74bb812cd        mcr.microsoft.com/azureiotedge-agent:1.0   "/bin/sh -c 'echo \"$…"   11 seconds ago      Up 4 seconds                            edgeAgent


Next we will proceed with notebook [031_DevAndRegisterModel.ipynb](./031_DevAndRegisterModel.ipynb).